# HPML HW-3
Aniket Mane(am14661)

In [2]:
!mkdir -p data
!wget -O data/movie-corpus.zip https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
!unzip -o data/movie-corpus.zip -d data

--2025-03-16 04:37:10--  https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
Resolving zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)... 128.253.51.179
Connecting to zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)|128.253.51.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40854701 (39M) [application/zip]
Saving to: ‘data/movie-corpus.zip’

data/movie-corpus.z 100%[===================>]  38.96M  9.03MB/s    in 4.3s    

2025-03-16 04:37:16 (9.03 MB/s) - ‘data/movie-corpus.zip’ saved [40854701/40854701]

Archive:  data/movie-corpus.zip
   creating: data/movie-corpus/
  inflating: data/movie-corpus/utterances.jsonl  
  inflating: data/movie-corpus/conversations.json  
  inflating: data/movie-corpus/corpus.json  
  inflating: data/movie-corpus/speakers.json  
  inflating: data/movie-corpus/index.json  


In [3]:
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json
import time
import numpy as np

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

Loading the data
dataset of movie character dialog:

220,579 conversational exchanges between

10,292 pairs of movie characters

9,035 characters from 617 movies

304,713 total utterances

In [4]:
corpus_name = "movie-corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "utterances.jsonl"))

b'{"id": "L1045", "conversation_id": "L1044", "text": "They do not!", "speaker": "u0", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "not", "tag": "RB", "dep": "neg", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": "L1044", "timestamp": null, "vectors": []}\n'
b'{"id": "L1044", "conversation_id": "L1044", "text": "They do to!", "speaker": "u2", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "to", "tag": "TO", "dep": "dobj", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": null, "timestamp": null, "vectors": []}\n'
b'{"id": "L985", "conversation_id": "L984", "text": "I hope so.", "speaker": "u0", "meta": {

In [5]:
# Splits each line of the file to create lines and conversations
def loadLinesAndConversations(fileName):
    lines = {}
    conversations = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            lineJson = json.loads(line)
            # Extract fields for line object
            lineObj = {}
            lineObj["lineID"] = lineJson["id"]
            lineObj["characterID"] = lineJson["speaker"]
            lineObj["text"] = lineJson["text"]
            lines[lineObj['lineID']] = lineObj

            # Extract fields for conversation object
            if lineJson["conversation_id"] not in conversations:
                convObj = {}
                convObj["conversationID"] = lineJson["conversation_id"]
                convObj["movieID"] = lineJson["meta"]["movie_id"]
                convObj["lines"] = [lineObj]
            else:
                convObj = conversations[lineJson["conversation_id"]]
                convObj["lines"].insert(0, lineObj)
            conversations[convObj["conversationID"]] = convObj

    return lines, conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [6]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict and conversations dict
lines = {}
conversations = {}
# Load lines and conversations
print("\nProcessing corpus into lines and conversations...")
lines, conversations = loadLinesAndConversations(os.path.join(corpus, "utterances.jsonl"))

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Processing corpus into lines and conversations...

Writing newly formatted file...

Sample lines from file:
b'They do to!\tThey do not!\n'
b'She okay?\tI hope so.\n'
b"Wow\tLet's go.\n"
b'"I\'m kidding.  You know how sometimes you just become this ""persona""?  And you don\'t know how to quit?"\tNo\n'
b"No\tOkay -- you're gonna need to learn how to lie.\n"
b"I figured you'd get to the good stuff eventually.\tWhat good stuff?\n"
b'What good stuff?\t"The ""real you""."\n'
b'"The ""real you""."\tLike my fear of wearing pastels?\n'
b'do you listen to this crap?\tWhat crap?\n'
b"What crap?\tMe.  This endless ...blonde babble. I'm like, boring myself.\n"


In [ ]:
# import json

# def loadLinesAndConversations(file_path):
#     lines = {}
#     conversations = {}

#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             try:
#                 data = json.loads(line.strip())  # Load each line as a separate JSON object
#                 lines[data["id"]] = data  # Store in dictionary
#                 conversation_id = data["conversation_id"]

#                 if conversation_id not in conversations:
#                     conversations[conversation_id] = []

#                 conversations[conversation_id].append(data["id"])
#             except json.JSONDecodeError as e:
#                 print("Skipping malformed line:", line)
#                 print("Error:", e)

#     return lines, conversations

# # Call the function
# file_path = os.path.join(corpus, "utterances.jsonl")
# lines, conversations = loadLinesAndConversations(file_path)

# # Debugging: Print a sample
# print("\nSample conversation mapping:")
# for convo_id, convo_lines in list(conversations.items())[:3]:  # Print first 3 conversations
#     print(f"Conversation {convo_id}: {convo_lines}")


KeyboardInterrupt: 

# Load and Trim Data

In [7]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

# Preprocessing
## 1)Unicode strings to ASCII using unicodeToAscii
## 2)convert all letters to lowercase and trim all non-letter characters except for basic punctuation (normalizeString)
## 3)filter out sentences with length greater than the MAX_LENGTH threshold (filterPairs).

In [8]:
MAX_LENGTH = 10  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True if both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using the ``filterPair`` condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64313 sentence pairs
Counting words...
Counted words: 18082

pairs:
['they do to !', 'they do not !']
['she okay ?', 'i hope so .']
['wow', 'let s go .']
['what good stuff ?', 'the real you .']
['the real you .', 'like my fear of wearing pastels ?']
['do you listen to this crap ?', 'what crap ?']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['have fun tonight ?', 'tons']


In [9]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7833 / 18079 = 0.4333
Trimmed from 64313 pairs to 53131, 0.8261 of total


# Prepare Data for Models


In [10]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  11,   24,   11,    9, 5714],
        [ 105,  136,  200,  895,    6],
        [  93,    5,    7,   14,    6],
        [ 511,  359, 1205, 2433,    2],
        [ 154,  160,  160, 6658,    0],
        [  24,  344,  161,   10,    0],
        [  13,  284,   72,    2,    0],
        [1497,   10,   14,    0,    0],
        [  14,    2,    2,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([10,  9,  9,  7,  4])
target_variable: tensor([[  67,  280,   19,  185,   19],
        [  17,   40,   10,  132,   17],
        [5914,   14,    2,   62,   22],
        [  10,    2,    0,  746,  342],
        [   2,    0,    0, 3932, 6542],
        [   0,    0,    0,   14,   10],
        [   0,    0,    0,    2,    2]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True, False,  True,  True],
        [ True, False, False,  True,  True],
        [False, 

#Models

Seq2seq Model

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size parameters are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=dropout, bidirectional=True)

    def forward(self, input_seq: torch.Tensor, input_lengths: torch.Tensor):
        hidden = torch.zeros(self.n_layers * 2, input_seq.shape[1], self.hidden_size).to(input_seq.device)
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

# Luong Attention Layer

In [12]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size, hidden_size)
        self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        attn_energies = torch.randn(1,1,1)

        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

# Decoder

In [13]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step: torch.Tensor, last_hidden: torch.Tensor, encoder_outputs: torch.Tensor):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

Define Training Procedure
=========================

Masked loss
-----------

Since we are dealing with batches of padded sequences, we cannot simply
consider all elements of the tensor when calculating loss. We define
`maskNLLLoss` to calculate our loss based on our decoder's output
tensor, the target tensor, and a binary mask tensor describing the
padding of the target tensor. This loss function calculates the average
negative log likelihood of the elements that correspond to a *1* in the
mask tensor.


In [14]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

# Single Training Iteration


## **Workflow of the Training Process**

```
┌──────────────────────────────────────────────────┐
│                  Input Sequence                  │
└──────────────────────────────────────────────────┘
                      │
                      ▼
┌──────────────────────────────────────────────────┐
│                 Encoder (RNN/GRU/LSTM)           │
└──────────────────────────────────────────────────┘
                      │
                      ▼
    ┌──────────────────────────────────────────┐
    │ Encoder Outputs + Final Hidden State (H) │
    └──────────────────────────────────────────┘
                      │
                      ▼
        ┌───────────────────────────────┐
        │ Decoder (RNN/GRU/LSTM)         │
        │ Initialized with Encoder's H   │
        └───────────────────────────────┘
                      │
                      ▼
 ┌─────────────────────────────────────────────────────────┐
 │ At each time step:                                      │
 │ - If Teacher Forcing: Use target word as next input.    │
 │ - Else: Use predicted word as next input.               │
 │ - Compute loss using maskNLLLoss().                     │
 └─────────────────────────────────────────────────────────┘
                      │
                      ▼
      ┌──────────────────────────────────────┐
      │ Compute total loss & Backpropagate   │
      └──────────────────────────────────────┘
                      │
                      ▼
      ┌──────────────────────────────────────┐
      │ Update Encoder & Decoder Parameters  │
      └──────────────────────────────────────┘
```
## **Pseudocode for Training Process**

```
1. Initialize encoder and decoder.
2. Pass input sequence through the encoder.
3. Store encoder outputs and hidden state.
4. Set decoder’s initial hidden state to encoder’s final state.
5. Loop over the sequence length:
   - If Teacher Forcing: Use target sequence as input.
   - Else: Use predicted word as input.
   - Compute loss using maskNLLLoss.
6. Compute total loss and backpropagate.
7. Update encoder and decoder weights.
```


In [15]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, teacher_forcing_ratio, max_length=MAX_LENGTH, record=False):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for RNN packing should always be on the CPU
    lengths = lengths.to('cpu')

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Recording loss on wandb
    if record:
        wandb.log({"loss": sum(print_losses) / n_totals})

    # Perform backpropagation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

# Training iterations

In [16]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, teacher_forcing_ratio, hidden_size, corpus_name, record=False):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip, teacher_forcing_ratio, record=record)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))


# Evaluation Process
## Greedy search Decoder


In [17]:

class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq: torch.Tensor, input_length: torch.Tensor, max_length: int):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self.decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, input_seq.shape[1], device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores


## Evaluate Text

In [18]:
def evaluate(searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to('cpu')
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words

# Evaluate inputs from user input (``stdin``)
def evaluateInput(searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

# Normalize input sentence and call ``evaluate()``
def evaluateExample(sentence, searcher, voc):
    print("> " + sentence)
    # Normalize sentence
    input_sentence = normalizeString(sentence)
    # Evaluate sentence
    output_words = evaluate(searcher, voc, input_sentence)
    output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    print('Bot:', ' '.join(output_words))

## Run Model


In [19]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

print('Intializing encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Intializing encoder and decoder ...
Models built and ready to go!


## Training

In [20]:
# Configure training/optimization parameters
clip = 50.0  # Gradient clipping threshold to avoid exploding gradients
teacher_forcing_ratio = 1.0  # The probability of using teacher forcing during training
learning_rate = 0.0001  # Learning rate for optimization
decoder_learning_ratio = 5.0  # Factor to scale decoder learning rate
n_iteration = 4000  # Total number of training iterations
print_every = 100  # Print training status every 100 iterations
save_every = 500  # Save model checkpoints every 500 iterations

# Ensure the encoder and decoder are in training mode
encoder.train()  # Set the encoder in training mode
decoder.train()  # Set the decoder in training mode

# Initialize the optimizers for the encoder and decoder
print('Intializing  optimizers ...')

# Use Adam optimizer for both encoder and decoder
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# If using CUDA (GPU), move optimizer states to GPU as well
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):  # Check if the parameter is a tensor
            state[k] = v.cuda()  # Move tensor to CUDA device

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):  # Check if the parameter is a tensor
            state[k] = v.cuda()  # Move tensor to CUDA device

# Start the training loop
print("Training Started !!")

# Train the models for the specified number of iterations
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, teacher_forcing_ratio, hidden_size, corpus_name)


Intializing  optimizers ...
Training Started !!
Initializing ...
Training...
Iteration: 100; Percent complete: 2.5%; Average loss: 5.0273
Iteration: 200; Percent complete: 5.0%; Average loss: 4.2915
Iteration: 300; Percent complete: 7.5%; Average loss: 4.0224
Iteration: 400; Percent complete: 10.0%; Average loss: 3.8815
Iteration: 500; Percent complete: 12.5%; Average loss: 3.7894
Iteration: 600; Percent complete: 15.0%; Average loss: 3.7101
Iteration: 700; Percent complete: 17.5%; Average loss: 3.6443
Iteration: 800; Percent complete: 20.0%; Average loss: 3.6088
Iteration: 900; Percent complete: 22.5%; Average loss: 3.5122
Iteration: 1000; Percent complete: 25.0%; Average loss: 3.4969
Iteration: 1100; Percent complete: 27.5%; Average loss: 3.4684
Iteration: 1200; Percent complete: 30.0%; Average loss: 3.4248
Iteration: 1300; Percent complete: 32.5%; Average loss: 3.3879
Iteration: 1400; Percent complete: 35.0%; Average loss: 3.3596
Iteration: 1500; Percent complete: 37.5%; Average los

In [21]:
# Set dropout layers to ``eval`` mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(searcher, voc)

> hi
Bot: hi . see you in the evening .
> what's up/
Bot: where is he ? go . ?
> quit


# Sweep


In [22]:
!pip install wandb --upgrade

In [23]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: samaniket9 (samaniket9-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [25]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'
    }

sweep_config['metric'] = metric
parameters = {
      'learning_rate': {
          'values': [0.0001, 0.00025, 0.0005, 0.001]
      },
      'optimizer': {
          'values': ["adam", "sgd"]
      },
      'clip': {
          'values': [0, 25, 50, 100]
      },
      'teacher_forcing_ratio': {
          'values': [0, 0.5, 1.0]
      },
      'decoder_learning_ratio':{
          'values': [1.0, 3.0, 5.0, 10.0]
      }
    }

sweep_config['parameters'] = parameters

In [26]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'clip': {'values': [0, 25, 50, 100]},
                'decoder_learning_ratio': {'values': [1.0, 3.0, 5.0, 10.0]},
                'learning_rate': {'values': [0.0001, 0.00025, 0.0005, 0.001]},
                'optimizer': {'values': ['adam', 'sgd']},
                'teacher_forcing_ratio': {'values': [0, 0.5, 1.0]}}}


In [ ]:
# def train_and_record():
#     # Initialize wandb run
#     run = wandb.init(project="W&BProjectName", entity="W&BUserName")
#     config = run.config

#     # Model architecture parameters
#     model_params = {
#         'name': 'cb_model',
#         'attn_model': 'dot',
#         'hidden_size': 500,
#         'encoder_layers': 2,
#         'decoder_layers': 2,
#         'dropout': 0.1,
#         'batch_size': 64
#     }

#     # Training parameters
#     training_params = {
#         'clip': config.clip,
#         'teacher_forcing_ratio': config.teacher_forcing_ratio,
#         'learning_rate': config.learning_rate,
#         'decoder_learning_ratio': config.decoder_learning_ratio,
#         'n_iteration': 4000,
#         'print_every': 1000,
#         'save_every': 5000
#     }

#     print('Building encoder and decoder ...')

#     # Initialize model components
#     embedding = nn.Embedding(voc.num_words, model_params['hidden_size'])
#     encoder = EncoderRNN(
#         model_params['hidden_size'],
#         embedding,
#         model_params['encoder_layers'],
#         model_params['dropout']
#     )
#     decoder = LuongAttnDecoderRNN(
#         model_params['attn_model'],
#         embedding,
#         model_params['hidden_size'],
#         voc.num_words,
#         model_params['decoder_layers'],
#         model_params['dropout']
#     )

#     # Move models to appropriate device
#     encoder = encoder.to(device)
#     decoder = decoder.to(device)
#     print('Models built and ready to go!')

#     # Set models to training mode
#     encoder.train()
#     decoder.train()

#     # Initialize optimizers
#     print('Building optimizers ...')
#     optimizers = initialize_optimizers(
#         config.optimizer,
#         encoder,
#         decoder,
#         training_params['learning_rate'],
#         training_params['decoder_learning_ratio']
#     )
#     encoder_optimizer, decoder_optimizer = optimizers

#     # Configure CUDA for optimizers if available
#     configure_cuda_for_optimizers(encoder_optimizer, decoder_optimizer)

#     # Run training iterations
#     print("Starting Training!")
#     trainIters(
#         model_params['name'],
#         voc,
#         pairs,
#         encoder,
#         decoder,
#         encoder_optimizer,
#         decoder_optimizer,
#         embedding,
#         model_params['encoder_layers'],
#         model_params['decoder_layers'],
#         save_dir,
#         training_params['n_iteration'],
#         model_params['batch_size'],
#         training_params['print_every'],
#         training_params['save_every'],
#         training_params['clip'],
#         training_params['teacher_forcing_ratio'],
#         model_params['hidden_size'],
#         corpus_name,
#         record=True
#     )

# def initialize_optimizers(optimizer_type, encoder, decoder, learning_rate, decoder_learning_ratio):
#     """Initialize optimizers based on the specified type"""
#     if optimizer_type == 'adam':
#         encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
#         decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
#     elif optimizer_type == 'sgd':
#         encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
#         decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
#     else:
#         raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

#     return encoder_optimizer, decoder_optimizer

# def configure_cuda_for_optimizers(encoder_optimizer, decoder_optimizer):
#     """Configure CUDA for optimizers if available"""
#     for optimizer in [encoder_optimizer, decoder_optimizer]:
#         for state in optimizer.state.values():
#             for k, v in state.items():
#                 if isinstance(v, torch.Tensor):
#                     state[k] = v.cuda()


In [27]:
def train_and_record():
    run = wandb.init(project="W&BProjectName", entity="W&BUserName")
    config = run.config

    # Configure models
    model_name = 'cb_model'
    attn_model = 'dot'
    hidden_size = 500
    encoder_n_layers = 2
    decoder_n_layers = 2
    dropout = 0.1
    batch_size = 64

    print('Building encoder and decoder ...')
    # Initialize word embeddings
    embedding = nn.Embedding(voc.num_words, hidden_size)

    # Initialize encoder & decoder models
    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)

    # Use appropriate device
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    print('Models built and ready to go!')

    # Configure training/optimization
    clip = config.clip
    teacher_forcing_ratio = config.teacher_forcing_ratio
    learning_rate = config.learning_rate
    decoder_learning_ratio = config.decoder_learning_ratio
    n_iteration = 4000
    # Changed to not populate the entire console with loss outputs
    print_every = 1000
    # Fix to NOT save model while running wandb
    save_every = 5000

    # Ensure dropout layers are in train mode
    encoder.train()
    decoder.train()

    # Initialize optimizers
    print('Building optimizers ...')
    if config.optimizer == 'adam':
        encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
    elif config.optimizer == 'sgd':
        encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

    # If you have CUDA, configure CUDA to call
    for state in encoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in decoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    # Run training iterations
    print("Starting Training!")
    trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
            embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
            print_every, save_every, clip, teacher_forcing_ratio, hidden_size, corpus_name, record=True)


In [28]:
sweep_id = wandb.sweep(sweep_config, project="Chabot-sweep")

Create sweep with ID: kdy3v9qq
Sweep URL: https://wandb.ai/samaniket9-new-york-university/Chabot-sweep/sweeps/kdy3v9qq


In [29]:
wandb.agent(sweep_id, function=train_and_record, count=25)

wandb: Agent Starting Run: vylmcnu6 with config:
wandb: 	clip: 25
wandb: 	decoder_learning_ratio: 3
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	teacher_forcing_ratio: 1


Building encoder and decoder ...
Models built and ready to go!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1000; Percent complete: 25.0%; Average loss: 3.9969
Iteration: 2000; Percent complete: 50.0%; Average loss: 3.3995
Iteration: 3000; Percent complete: 75.0%; Average loss: 3.1509
Iteration: 4000; Percent complete: 100.0%; Average loss: 2.9707


loss,█▄▄▄▃▃▃▂▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▁▁▁
loss,2.67114


wandb: Agent Starting Run: q2ka8t8c with config:
wandb: 	clip: 25
wandb: 	decoder_learning_ratio: 3
wandb: 	learning_rate: 0.00025
wandb: 	optimizer: sgd
wandb: 	teacher_forcing_ratio: 0


Building encoder and decoder ...
Models built and ready to go!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1000; Percent complete: 25.0%; Average loss: 6.1593
Iteration: 2000; Percent complete: 50.0%; Average loss: 5.1465
Iteration: 3000; Percent complete: 75.0%; Average loss: 4.9530
Iteration: 4000; Percent complete: 100.0%; Average loss: 4.8733


...

Building encoder and decoder ...
Models built and ready to go!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1000; Percent complete: 25.0%; Average loss: 5.5761
Iteration: 2000; Percent complete: 50.0%; Average loss: 5.8221
Iteration: 3000; Percent complete: 75.0%; Average loss: 5.6207
Iteration: 4000; Percent complete: 100.0%; Average loss: 5.6711


loss,▄▃▂▂▄▂▄▃▂█▅▅▇▅▅▄▃▃▆▂▅▃▃▂▆▄▄▅▃▃▃▂▆▄▃▄▃▂▁▁
loss,5.24184


wandb: Agent Starting Run: mqqny37e with config:
wandb: 	clip: 0
wandb: 	decoder_learning_ratio: 5
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	teacher_forcing_ratio: 1


Building encoder and decoder ...
Models built and ready to go!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1000; Percent complete: 25.0%; Average loss: 8.9703
Iteration: 2000; Percent complete: 50.0%; Average loss: 8.9703
Iteration: 3000; Percent complete: 75.0%; Average loss: 8.9703
Iteration: 4000; Percent complete: 100.0%; Average loss: 8.9703


loss,▆▅▄▅▄▄▅▅▅▅▄▅▆▅▄▄▆▅▆▄▆▃▇▅▆▁▆▆▅▅█▄▄▃▄▅█▇▅▇
loss,8.96745


# Best Model (1.5)
loss	2.27358


wandb: Agent Starting Run: ocu50ucj with config:

clip: 50

decoder_learning_ratio: 3

learning_rate: 0.00025

optimizer: adam

teacher_forcing_ratio: 1

In [30]:
def run_best_model(clip=50, decoder_learning_ratio=3, learning_rate=0.00025, optimizer='adam', teacher_forcing_ratio=1):
    # Configure models
    model_name = 'cb_model'
    attn_model = 'dot'
    hidden_size = 500
    encoder_n_layers = 2
    decoder_n_layers = 2
    dropout = 0.1
    batch_size = 64

    print('Building encoder and decoder ...')
    # Initialize word embeddings
    embedding = nn.Embedding(voc.num_words, hidden_size)

    # Initialize encoder & decoder models
    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)

    # Use appropriate device
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    print('Models built and ready to go!')

    # Configure training/optimization
    clip = clip
    teacher_forcing_ratio = teacher_forcing_ratio
    learning_rate = learning_rate
    decoder_learning_ratio = decoder_learning_ratio
    n_iteration = 4000
    # Changed to not populate the entire console with loss outputs
    print_every = 500
    save_every = 1000

    # Ensure dropout layers are in train mode
    encoder.train()
    decoder.train()

    # Initialize optimizers
    print('Building optimizers ...')
    if optimizer == 'adam':
        encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
    elif optimizer == 'sgd':
        encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

    # If you have CUDA, configure CUDA to call
    for state in encoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in decoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    # Run training iterations
    print("Starting Training!")
    trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
            embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
            print_every, save_every, clip, teacher_forcing_ratio, hidden_size, corpus_name)

    return encoder, decoder

In [31]:
best_model_enc, best_model_dec = run_best_model()

Building encoder and decoder ...
Models built and ready to go!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 500; Percent complete: 12.5%; Average loss: 4.1209
Iteration: 1000; Percent complete: 25.0%; Average loss: 3.5202
Iteration: 1500; Percent complete: 37.5%; Average loss: 3.2869
Iteration: 2000; Percent complete: 50.0%; Average loss: 3.0775
Iteration: 2500; Percent complete: 62.5%; Average loss: 2.8703
Iteration: 3000; Percent complete: 75.0%; Average loss: 2.6745
Iteration: 3500; Percent complete: 87.5%; Average loss: 2.4702
Iteration: 4000; Percent complete: 100.0%; Average loss: 2.2845


# Problem 1.6-1.7

In [32]:
from torch.profiler import profile, record_function, ProfilerActivity

In [33]:
best_model_enc.eval()
best_model_dec.eval()

input_sentence = 'hello how are you?'
print('> ' + input_sentence)

input_sentence = normalizeString(input_sentence)
best_model_searcher = GreedySearchDecoder(best_model_enc, best_model_dec)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof:
    with record_function("model_inference"):
        output_words = evaluate(best_model_searcher, voc, input_sentence)
        output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
        print('Bot:', ' '.join(output_words))

> hello how are you?
Bot: okay . thank you . . .


In [34]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        66.82%      37.291ms        99.96%      55.784ms      55.784ms       0.000us         0.00%       4.315ms       4.315ms           0 b      -7.96 Kb           0 b      -1.03 M

In [35]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls                                                                      Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------------------------------------------------------------------

In [36]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      98.884ms      2293.48%      98.884ms      10.987ms           0 b           0 b           0 b           0 

In [37]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::empty         1.32%     735.336us         1.32%     735.336us       6.339us       0.000us         0.00%       0.000us       0.000us       7.96 Kb       7.96 Kb     364.85 Mb     364.85 M

In [38]:
print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::empty         1.32%     735.336us         1.32%     735.336us       6.339us       0.000us         0.00%       0.000us       0.000us       7.96 Kb       7.96 Kb     364.85 Mb     364.85 M

In [39]:
prof.export_chrome_trace("trace.json")

Problem 2.3

In [40]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder, decoder_n_layers):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self._device = device
        self._SOS_token = SOS_token
        self._decoder_n_layers = decoder_n_layers

    __constants__ = ['_device', '_SOS_token', '_decoder_n_layers']

    def forward(self, input_seq: torch.Tensor, input_length: torch.Tensor, max_length: int):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self._decoder_n_layers]
        # Initialize decoder input with SOS_token

        decoder_input = torch.ones(1, input_seq.shape[1], device=self._device, dtype=torch.long) * self._SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=self._device, dtype=torch.long)
        all_scores = torch.zeros([0], device=self._device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

# TorchScript

In [41]:
device = torch.device('cpu')

In [42]:
save_dir = os.path.join("data", "save")
corpus_name = "movie-corpus"

# Configure models
model_name = 'cb_model'
attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

checkpoint_iter = 4000

loadFilename = os.path.join(save_dir, model_name, corpus_name,
                         '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                         '{}_checkpoint.tar'.format(checkpoint_iter))

# Load model
# Force CPU device options (to match tensors in this tutorial)
checkpoint = torch.load(loadFilename, map_location=device)
encoder_sd = checkpoint['en']
decoder_sd = checkpoint['de']
encoder_optimizer_sd = checkpoint['en_opt']
decoder_optimizer_sd = checkpoint['de_opt']
embedding_sd = checkpoint['embedding']
voc = Voc(corpus_name)
voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
# Load trained model parameters
encoder.load_state_dict(encoder_sd)
decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
# Set dropout layers to ``eval`` mode
encoder.eval()
decoder.eval()
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [43]:
test_seq = torch.LongTensor(MAX_LENGTH, 1).random_(0, voc.num_words).to(device)
test_seq_length = torch.LongTensor([test_seq.size()[0]]).to(torch.device('cpu'))
# Trace the model
traced_encoder = torch.jit.trace(encoder, (test_seq, test_seq_length))

### Convert decoder model
# Create and generate artificial inputs
test_encoder_outputs, test_encoder_hidden = traced_encoder(test_seq, test_seq_length)
test_decoder_hidden = test_encoder_hidden[:decoder.n_layers]
test_decoder_input = torch.LongTensor(1, 1).random_(0, voc.num_words)

# Move the test inputs to the same device as the model (GPU)
test_encoder_outputs = test_encoder_outputs.to(device)
test_decoder_hidden = test_decoder_hidden.to(device)
test_decoder_input = test_decoder_input.to(device)

# Trace the model
traced_decoder = torch.jit.trace(decoder, (test_decoder_input, test_decoder_hidden, test_encoder_outputs))

### Initialize searcher module by wrapping ``torch.jit.script`` call
scripted_searcher = torch.jit.script(GreedySearchDecoder(traced_encoder, traced_decoder, decoder.n_layers))



/usr/local/lib/python3.11/dist-packages/torch/jit/_trace.py:165: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  if a.grad is not None:


Problem 2.4

In [44]:
print('scripted_searcher graph:\n', scripted_searcher.graph)

scripted_searcher graph:
 graph(%self : __torch__.GreedySearchDecoder,
      %input_seq.1 : Tensor,
      %input_length.1 : Tensor,
      %max_length.1 : int):
  %56 : bool = prim::Constant[value=0]()
  %45 : bool = prim::Constant[value=1]() # <ipython-input-40-22c2fc12193e>:24:8
  %21 : int = prim::Constant[value=4]() # <ipython-input-40-22c2fc12193e>:19:85
  %20 : Device = prim::Constant[value="cpu"]() # <ipython-input-40-22c2fc12193e>:19:65
  %14 : NoneType = prim::Constant()
  %12 : int = prim::Constant[value=2]() # <ipython-input-40-22c2fc12193e>:16:41
  %16 : int = prim::Constant[value=1]() # <ipython-input-40-22c2fc12193e>:19:35
  %29 : int = prim::Constant[value=0]() # <ipython-input-40-22c2fc12193e>:21:34
  %encoder : __torch__.EncoderRNN = prim::GetAttr[name="encoder"](%self)
  %7 : (Tensor, Tensor) = prim::CallMethod[name="forward"](%encoder, %input_seq.1, %input_length.1) # <ipython-input-40-22c2fc12193e>:14:42
  %encoder_outputs.1 : Tensor, %encoder_hidden.1 : Tensor = pri

Problem 2.5


In [45]:
scripted_searcher.to(device)
# Set dropout layers to ``eval`` mode
scripted_searcher.eval()

# Evaluate examples
sentences = ["hello", "what's up?","How are you ?" , "where am I?", "where are you from?"]
for s in sentences:
    evaluateExample(s, scripted_searcher, voc)

> hello
Bot: i m ready to get you back . .
> what's up?
Bot: i m thirsty . . . .
> How are you ?
Bot: okay . i m okay . . .
> where am I?
Bot: in the house . . . .
> where are you from?
Bot: southern . . . . .


Problem 2.6

In [46]:
def evaluateAndTime(searcher, eval_batches, n_iteration, device, max_length=MAX_LENGTH):
    time_diffs = []
    searcher = searcher.to(device)
    for iteration in range(n_iteration):
        input_variable, lengths, target_variable, mask, max_target_len = eval_batches[iteration]
        input_variable = input_variable.to(device)
        lengths = lengths.to('cpu')
        if iteration > 5:
            torch.cuda.synchronize()
            start = time.monotonic_ns()
        with torch.no_grad():
            output = searcher(input_variable, lengths, max_length)
        if iteration > 5:
            torch.cuda.synchronize()
            end = time.monotonic_ns()
        if iteration > 5:
            time_diffs.append(end - start)
    return np.array(time_diffs) / 1_000_000

In [47]:
n_iteration = 100

eval_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(64)])
                    for _ in range(n_iteration)]

In [48]:
device = torch.device('cuda')

regular_gpu = evaluateAndTime(best_model_searcher, eval_batches, n_iteration, device)

print(regular_gpu.mean())

10.70643210638298


In [49]:
device = torch.device('cpu')

regular_cpu = evaluateAndTime(best_model_searcher, eval_batches, n_iteration, device)

print(regular_cpu.mean())

287.02651021276597


In [50]:
device = torch.device('cpu')

torchscript_cpu = evaluateAndTime(scripted_searcher, eval_batches, n_iteration, device)

print(torchscript_cpu.mean())

255.13287111702127


In [51]:
scripted_searcher.save("scripted_chatbot_cpu.pth")

# TorchScript Model to GPU

In [52]:
device = torch.device('cuda')

save_dir = os.path.join("data", "save")
corpus_name = "movie-corpus"

# Configure models
model_name = 'cb_model'
attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

checkpoint_iter = 4000

loadFilename = os.path.join(save_dir, model_name, corpus_name,
                         '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                         '{}_checkpoint.tar'.format(checkpoint_iter))

# Load model
# Force CPU device options (to match tensors in this tutorial)
checkpoint = torch.load(loadFilename, map_location=device)
encoder_sd = checkpoint['en']
decoder_sd = checkpoint['de']
encoder_optimizer_sd = checkpoint['en_opt']
decoder_optimizer_sd = checkpoint['de_opt']
embedding_sd = checkpoint['embedding']
voc = Voc(corpus_name)
voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
# Load trained model parameters
encoder.load_state_dict(encoder_sd)
decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
# Set dropout layers to ``eval`` mode
encoder.eval()
decoder.eval()
print('Models built and ready to go!')

### Compile the whole greedy search model to TorchScript model
# Create artificial inputs
test_seq = torch.LongTensor(MAX_LENGTH, 1).random_(0, voc.num_words).to(device)
test_seq_length = torch.LongTensor([test_seq.size()[0]]).to(torch.device('cpu'))
# Trace the model
traced_encoder = torch.jit.trace(encoder, (test_seq, test_seq_length))

### Convert decoder model
# Create and generate artificial inputs
test_encoder_outputs, test_encoder_hidden = traced_encoder(test_seq, test_seq_length)
test_decoder_hidden = test_encoder_hidden[:decoder.n_layers]
test_decoder_input = torch.LongTensor(1, 1).random_(0, voc.num_words)

# Move the test inputs to the same device as the model (GPU)
test_encoder_outputs = test_encoder_outputs.to(device)
test_decoder_hidden = test_decoder_hidden.to(device)
test_decoder_input = test_decoder_input.to(device)

# Trace the model
traced_decoder = torch.jit.trace(decoder, (test_decoder_input, test_decoder_hidden, test_encoder_outputs))

### Initialize searcher module by wrapping ``torch.jit.script`` call
scripted_searcher = torch.jit.script(GreedySearchDecoder(traced_encoder, traced_decoder, decoder.n_layers))

print('scripted_searcher graph:\n', scripted_searcher.graph)

Building encoder and decoder ...
Models built and ready to go!
scripted_searcher graph:
 graph(%self : __torch__.___torch_mangle_37.GreedySearchDecoder,
      %input_seq.1 : Tensor,
      %input_length.1 : Tensor,
      %max_length.1 : int):
  %56 : bool = prim::Constant[value=0]()
  %45 : bool = prim::Constant[value=1]() # <ipython-input-40-22c2fc12193e>:24:8
  %21 : int = prim::Constant[value=4]() # <ipython-input-40-22c2fc12193e>:19:85
  %20 : Device = prim::Constant[value="cuda"]() # <ipython-input-40-22c2fc12193e>:19:65
  %14 : NoneType = prim::Constant()
  %12 : int = prim::Constant[value=2]() # <ipython-input-40-22c2fc12193e>:16:41
  %16 : int = prim::Constant[value=1]() # <ipython-input-40-22c2fc12193e>:19:35
  %29 : int = prim::Constant[value=0]() # <ipython-input-40-22c2fc12193e>:21:34
  %encoder : __torch__.___torch_mangle_17.EncoderRNN = prim::GetAttr[name="encoder"](%self)
  %7 : (Tensor, Tensor) = prim::CallMethod[name="forward"](%encoder, %input_seq.1, %input_length.1) #

In [53]:
device = torch.device('cuda')

torchscript_gpu = evaluateAndTime(scripted_searcher, eval_batches, n_iteration, device)

print(torchscript_gpu.mean())

9.385404382978722


In [54]:
import pandas as pd

timing_table = pd.DataFrame({
    'Framework': ['PyTorch', 'TorchScript'],
    'Latency on CPU (ms)': [regular_cpu.mean(), torchscript_cpu.mean()],
    'Latency on GPU (ms)': [regular_gpu.mean(), torchscript_gpu.mean()]
})

In [55]:
timing_table

,Framework,Latency on CPU (ms),Latency on GPU (ms)
0,PyTorch,287.026510,10.706432
1,TorchScript,255.132871,9.385404


In [58]:
scripted_searcher.save("scripted_chatbot_gpu.pth")